<a href="https://colab.research.google.com/github/timeseriesAI/tsai/blob/master/tutorial_nbs/05_TS_archs_comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

created by Ignacio Oguiza - email: oguiza@timeseriesAI.co

# Import libraries 📚

In [ ]:
# # **************** UNCOMMENT AND RUN THIS CELL IF YOU NEED TO INSTALL/ UPGRADE TSAI ****************
# stable = True # Set to True for latest pip version or False for main branch in GitHub
# !pip install {"tsai -U" if stable else "git+https://github.com/timeseriesAI/tsai.git"} >> /dev/null

In [ ]:
from tsai.all import *
from IPython.display import clear_output
my_setup()

os              : Linux-5.10.133+-x86_64-with-Ubuntu-18.04-bionic
python          : 3.7.15
tsai            : 0.3.5
fastai          : 2.7.10
fastcore        : 1.5.27
torch           : 1.12.1+cu113
device          : 1 gpu (['Tesla T4'])
cpu cores       : 1
threads per cpu : 2
RAM             : 12.68 GB
GPU memory      : [14.75] GB


# Experiments 🧪

I've run a small test to show how you can build any model in the `tsai` library using the `create_model` functions.

The esiest way to do it is pass the architecture you want to use, a `TSDataLoaders` and any `kwargs` you'd like to use. The `create_model` function will pick up the necessary data from the `TSDataLoaders` object.

I've used 2 multivariate time series datasets. As you can see, it's difficult to predict which architecture will work best for which dataset.

Please, bear in mind that this is a very simple test without any hyperparameter tuning.

In [ ]:
dsid = 'NATOPS' 
bs = 64
X, y, splits = get_UCR_data(dsid, return_split=False)
print(X.shape)
tfms  = [None, [Categorize()]]
dsets = TSDatasets(X, y, tfms=tfms, splits=splits)
dls   = TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=[bs, bs*2])

archs = [(FCN, {}), (ResNet, {}), (xresnet1d34, {}), (ResCNN, {}), 
         (LSTM, {'n_layers':1, 'bidirectional': False}), (LSTM, {'n_layers':2, 'bidirectional': False}), (LSTM, {'n_layers':3, 'bidirectional': False}), 
         (LSTM, {'n_layers':1, 'bidirectional': True}), (LSTM, {'n_layers':2, 'bidirectional': True}), (LSTM, {'n_layers':3, 'bidirectional': True}),
         (LSTM_FCN, {}), (LSTM_FCN, {'shuffle': False}), (InceptionTime, {}), (XceptionTime, {}), (OmniScaleCNN, {}), (mWDN, {'levels': 4})]

results = pd.DataFrame(columns=['arch', 'hyperparams', 'total params', 'train loss', 'valid loss', 'accuracy', 'time'])
for i, (arch, k) in enumerate(archs):
    model = create_model(arch, dls=dls, **k)
    print(model.__class__.__name__)
    learn = Learner(dls, model,  metrics=accuracy)
    start = time.time()
    learn.fit_one_cycle(100, 1e-3)
    elapsed = time.time() - start
    vals = learn.recorder.values[-1]
    results.loc[i] = [arch.__name__, k, count_parameters(model), vals[0], vals[1], vals[2], int(elapsed)]
    results.sort_values(by='accuracy', ascending=False, kind='stable', ignore_index=True, inplace=True)
    clear_output()
    display(results)

,arch,hyperparams,total params,train loss,valid loss,accuracy,time
0,LSTM_FCN,{},347246,0.074354,0.116819,0.966667,6
1,FCN,{},285446,0.074158,0.125400,0.961111,5
2,LSTM_FCN,{'shuffle': False},336446,0.066596,0.113484,0.961111,6
3,XceptionTime,{},403420,0.399169,0.499384,0.961111,9
4,ResNet,{},490758,0.025244,0.122984,0.955556,7
5,ResCNN,{},268551,0.041432,0.134848,0.950000,6
6,InceptionTime,{},460038,0.026669,0.113258,0.938889,9
7,mWDN,{'levels': 4},467038,0.025454,0.274648,0.927778,9
8,xresnet1d34,{},7232518,0.022142,0.318501,0.916667,15
9,OmniScaleCNN,{},5239596,0.193854,0.260377,0.883333,19


In [ ]:
dsid = 'LSST' 
bs = 64
X, y, splits = get_UCR_data(dsid, return_split=False)
print(X.shape)
tfms  = [None, [Categorize()]]
dsets = TSDatasets(X, y, tfms=tfms, splits=splits)
dls   = TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=[bs, bs*2])

archs = [(FCN, {}), (ResNet, {}), (xresnet1d34, {}), (ResCNN, {}), 
         (LSTM, {'n_layers':1, 'bidirectional': False}), (LSTM, {'n_layers':2, 'bidirectional': False}), (LSTM, {'n_layers':3, 'bidirectional': False}), 
         (LSTM, {'n_layers':1, 'bidirectional': True}), (LSTM, {'n_layers':2, 'bidirectional': True}), (LSTM, {'n_layers':3, 'bidirectional': True}),
         (LSTM_FCN, {}), (LSTM_FCN, {'shuffle': False}), (InceptionTime, {}), (XceptionTime, {}), (OmniScaleCNN, {}), (mWDN, {'levels': 4})]

results = pd.DataFrame(columns=['arch', 'hyperparams', 'total params', 'train loss', 'valid loss', 'accuracy', 'time'])
for i, (arch, k) in enumerate(archs):
    model = create_model(arch, dls=dls, **k)
    print(model.__class__.__name__)
    learn = Learner(dls, model,  metrics=accuracy)
    start = time.time()
    learn.fit_one_cycle(100, 1e-3)
    elapsed = time.time() - start
    vals = learn.recorder.values[-1]
    results.loc[i] = [arch.__name__, k, count_parameters(model), vals[0], vals[1], vals[2], int(elapsed)]
    results.sort_values(by='accuracy', ascending=False, kind='stable', ignore_index=True, inplace=True)
    clear_output()
    display(results)

,arch,hyperparams,total params,train loss,valid loss,accuracy,time
0,XceptionTime,{},401580,0.070966,1.596494,0.680049,92
1,ResCNN,{},260367,0.231670,1.220587,0.677210,56
2,LSTM,"{'n_layers': 3, 'bidirectional': True}",572414,0.000120,2.694554,0.669100,50
3,LSTM_FCN,{'shuffle': False},314950,0.416285,1.403729,0.662206,45
4,LSTM,"{'n_layers': 2, 'bidirectional': True}",330814,0.000262,2.415106,0.657745,44
5,ResNet,{},482574,0.021112,1.656618,0.653690,71
6,LSTM,"{'n_layers': 2, 'bidirectional': False}",125414,0.002257,2.448107,0.639092,35
7,LSTM,"{'n_layers': 3, 'bidirectional': False}",206214,0.001700,2.582463,0.632198,39
8,LSTM,"{'n_layers': 1, 'bidirectional': True}",89214,0.056981,1.748464,0.606650,34
9,LSTM,"{'n_layers': 1, 'bidirectional': False}",44614,0.058143,1.877555,0.601379,31
